# Machine learning III

files needed = ('Hitters.csv')

This book continues our foray into machine learning. Our goals here are modest. We would like to
1. learn a bit about how machine learning is similar to, and different from, econometrics.  
2. introduce the scikit-learn package which is chock full of 'machine learning' tools. 
3. work on some *validation* methods, which are an important part of the machine learning toolkit. 
4. explore the ridge and lasso regression models

In this notebook, we study the ridge and lasso regressions. These are methods that put discipline on the importance of the independent variables of the regression.

This notebook is loosely based on Chapter 6 from *An Introduction to Statistical Learning* by James, Witten, Hastie, and Tibshirani. This is an easy to follow introduction that is light on the mathematics behind the methods.

## Econometrics v. Statistical learning
This is overly broad and general, but hopefully helpful. Consider the model

$$ y = f(X) + \epsilon.$$

### Econometrics
Econometrics is mainly concerned with *inference*. By this, we mean that the goal is to understand the structure of $f(\;)$. Econometricians are concerned about the 'true' value of the components of $f(\;)$. We worry a lot about endogeneity, omitted variables, and whether the properties of $f(\;)$ are consistent with the theory.  Practically, 

1. The $X$ variables included in the model are guided by theory.
2. The focus is on in-sample fit. How well does the model fit the data?

### Statistical (machine) learning
Statistical learning is mainly concerned with *prediction*. By this, we mean the ability of the model to predict the values of $y$, given $X$, for data that are not used to estimate (or, in machine learning-ese, train) the model of $f(\;)$. The guiding principle here is the *bias-variance tradeoff*. \[more on this in a minute\]. Practically, 

1. The $X$ variables included in the model are guided by the mean-variance tradeoff. 
2. The focus is on out-of-sample fit. How well does the model predict data not used to estimate the model?

\[The bias-variance tradeoff exist in econometrics, too. Theory typically disciplines our definition of $X$.



### Evaluating model predictions

The fundamental constraint in machine learning is the *bias-variance tradeoff*. Roughly speaking, there is no free lunch. 

Let's start with an example. Suppose we wanted to predict the number of people on the union terrace on a Friday. For 5 Fridays, we send out a team to count the number of people on the terrace (the $y$ variable). We also record the temperature, the price of beer, if it is a home football weekend, the value of the stock market, the number of sailboats on Mendota, and the euro-dollar exchange rate (the $X$ variables).  

We use the data to estimate our model $y=f(X)+\epsilon$ and use the $X$ data for a 6th Friday to predict the number of people on the terrace, $\hat{y}$. We then evaluate our estimate by comparing our prediction to the actual number of people on the terrace on the 6th Friday, $(y - \hat{y})^2$. This is a measure of how well our model works at predicting **out of sample**. 

Now, suppose we repeat this experiment $M$ times. We collect $M$ data sets, estimate the model $M$ times, predict the 6th Friday $M$ times. We can form the expected test mse as

$$\frac{1}{M}\sum_{m=1}^M(y_m-\hat{y}_m)^2$$

It is straightforward to show that this expression can be decomposed into 

$$\begin{align*}
\frac{1}{M}\sum_{m=1}^M(y_m-\hat{y}_m)^2 & = \left(E\left[\,\hat{f}(X)\right]-f(X) \right)^2 + E\,\left(\,\hat{f}(X)-E\left[\,\hat{f}(X)\right]\right)^2 + \text{var}(\epsilon)\\
                                         &= \text{bias}^2 + \text{variance} + \text{var}(\epsilon).\\
\end{align*}$$

Note that all three terms are positive. We can't do anything about the third term, it is the *irreducible* error. The other two terms we would like to make as small as possible. Unfortunately, shrinking one of the two terms tends to increase the other term. 

### Bias and variance
The **bias term** tells us, on average, how close we get to the true model. Are we systematically over- or under-predicting $y$?

The **variance term** tells us how much our estimates vary as we use different training datasets to estimate the model.

Quick check: Think about shooting arrows at a target. What does a low-variance high-bias attempt look like? A low-bias low-variance look like?

### The bias-variance tradeoff

We would like to minimize the bias and the variance of the test mse. How can we do so? 

In the linear models we are considering, complexity increases as we add more variables to $X$. This includes adding polynomials of our independent variables, interactions, etc. How does complexity influence the testing error?  

* As we **increase the complexity** of our model $f(\;)$ the **squared bias tends to decrease**. 

* As we **increase the complexity** of our model $f(\;)$ the **variance tends to increase**.

This is the tradeoff. As we add features to the model, the bias decreases, but the variance increases. This gives rise to a u-shaped mse. This [figure](http://www-bcf.usc.edu/~gareth/ISL/Chapter2/2.12.pdf) from James et al. is a good illustration. 

### Overfitting

Behind the bias-variance tradoff is the idea of overfitting. The more complex the model, the more it will capture variation in $y$ due to the random error ($\epsilon$). 

* This makes the model fit the data better (lower bias). We are capturing $y$ behavior from both $f(\;)$ and $\epsilon$.
* This makes the model more variable. A new set of training data will have different $\epsilon$'s. The estimate will change to match these new values of $\epsilon$. Since $\epsilon$ is not related to $f(\;)$, we are making the estimate noisier.

## Shrinkage methods
The bias-variance tradeoff says that we want to constrain our model's complexity. There are many, many, many ways to go about this. For linear models, two common and easy to grasp methods are the ridge and the lasso regression. 

Let's see how they work. 

In [ ]:
import pandas as pd                 # data handling
import numpy as np                  # numerical methods
import matplotlib.pyplot as plt     # plots

Load data on baseball players. Each row is a player. The variable we would like to predict is salary.

In [ ]:
base = pd.read_csv('Hitters.csv')
print(base.info())

In [ ]:
base.head(2)

The data look okay, but we only have salary for 263 observations. Let's drop them. 

In [ ]:
base = base.dropna()
base.info()

### OLS

Let's start with ols to get a feel for things. Start by loading some packages.

In [ ]:
from sklearn import linear_model                          # ols, ridge, lasso, 
from sklearn.preprocessing import StandardScaler          # normalize variables to have mu=0, std=1

Let's choose some variables that are potentially useful for predicting salary. We are purposely making this set large. The goal is determine how to constrain our choices. 

The ridge regression works best if the $X$ variables are on the same scale. The `StandardScaler()` normalizes the variables.

In [ ]:
var_list = ['Hits', 'RBI', 'HmRun', 'Walks', 'Errors', 'Years', 'Assists', 'AtBat', 'Runs', 'CAtBat', 'CHits', 'CRuns', 'CWalks']

# Standardize the X vars so they have mean = 0 and std = 1
X = StandardScaler().fit_transform(base[var_list])

Estimate the OLS regression. Don't worry about the l2 norm stuff yet. 

In [ ]:
res_ols = linear_model.LinearRegression().fit(X, base['Salary'])
coef_norm_ols = np.linalg.norm(res_ols.coef_, ord=2)

print(res_ols.coef_)
print('The l2 norm of the ols coefficients is {0:5.1f}.'.format(coef_norm_ols))

## The ridge regression

The ridge regression chooses $\beta$ to minimize the residual sum of squares plus a penalty function 

$$
\begin{align*}
=& \text{RSS}+ \alpha \left(\sum_{j=1}^p \beta_j^2\right)^{0.5}\\
=&\sum_{i=1}^n(y_i-\hat{y}_i)^2 + \alpha \left(\sum_{j=1}^p \beta_j^2\right)^{0.5}
\end{align*}
$$

OLS minimizes RSS, so if $\alpha=0$ ridge collapses to OLS. We call $\alpha$ the tuning parameter. When $\alpha>0$, models are penalized for how large their coefficients are. The term multiplying $\alpha$ is the l2 norm of the coefficient vector.  

The `Ridge()` function is part of `linear_models` in scikit-learn [(docs)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). 

First, let's estimate the model with $\alpha=0$. This should return the ols estimate. 

In [ ]:
res_ridge_0 = linear_model.Ridge(alpha = 0.0).fit(X, base['Salary'])
coef_norm_r0 = np.linalg.norm(res_ridge_0.coef_, ord=2)

print(res_ridge_0.coef_)
print('The l2 norm of the ridge(a=0) coefficients is {0:5.1f}.'.format(coef_norm_r0))

Now estimate the ridge model with $\alpha=100$. This adds a penalty to each coefficient that is not zero. 

In [ ]:
res_ridge_100 = linear_model.Ridge(alpha = 100.0).fit(X, base['Salary'])
coef_norm_r100 = np.linalg.norm(res_ridge_100.coef_, ord=2)

print(res_ridge_100.coef_)
print('The l2 norm of the ridge(a=100) coefficients is {0:5.1f}.'.format(coef_norm_r100))

That decreased the norm of the coefficients quite a bit. We can keep going... 

In [ ]:
res_ridge_800 = linear_model.Ridge(alpha = 800.0).fit(X, base['Salary'])
coef_norm_r800 = np.linalg.norm(res_ridge_800.coef_, ord=2)

print(res_ridge_800.coef_)
print('The l2 norm of the ridge(a=800) coefficients is {0:5.1f}.'.format(coef_norm_r800))


In [ ]:
pd.DataFrame({'var': var_list, 'ols': res_ols.coef_ , 'ridge 100':res_ridge_100.coef_, 'ridge 800':res_ridge_800.coef_ })

### What size penalty? 

We see that increasing $\alpha$ decreases the norm of the coefficients. How big should $\alpha$ be? We want the $\alpha$ that gives us the best test mse. As we discussed last week, cross-validation methods allow us to evaluate test mses.

The scikit package has a method that combines the ridge estimation with cross validation. It is called `RidgeCV()` [(docs)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

We pass to `RidgeCV()` a list of the alpha values to try. 

In [ ]:
alpha_grid = [1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4]

# Setting 'store_cv_values' to true will hang on to all the test mses from the CV. Otherwise, it only keep the best one.
model = linear_model.RidgeCV(alphas=alpha_grid, store_cv_values = True).fit(X,base['Salary'])

The function returns an object with useful attributes and methods. Let's look at a few.

In [ ]:
# The .alpha_ holds the best alpha
print('The best alpha from the candidate alphas is {0}.'.format(model.alpha_))

In [ ]:
best_coef_ridge = pd.DataFrame({'var':var_list, 'coef': model.coef_}) 
print(best_coef_ridge)

In [ ]:
# Since I set 'store_cv_values' to true, I have the matrix of all the test mse. Columns correspond to alpha values, 
# and there is one row for each observation, since the function uses loocv

print(model.cv_values_)
print(model.cv_values_.shape)

In [ ]:
# The mean test mse for each value of alpha
model.cv_values_.mean(axis=0)

## The lasso regression
The lasso regression works like the ridge, but with a different penalty function. 

$$
\begin{align*}
=& \text{RSS}+ \alpha \sum_{j=1}^p | \beta_j|\\
=&\sum_{i=1}^n(y_i-\hat{y}_i)^2 + \alpha \sum_{j=1}^p | \beta_j|
\end{align*}
$$

The penalty function here is the l1 norm, or the sum of the absolute values of the absolute values of the coefficient. The major difference between the ridge and the lasso is that the lasso can generate coefficients that are zero, dropping them from the model and making it simpler.

The `Lasso` regression is part of `linear_models` in scikit-learn [(docs)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html).   

In [ ]:
res_lasso_0 = linear_model.Lasso(alpha = 0.0).fit(X, base['Salary'])
coef_norm_l0 = np.linalg.norm(res_lasso_0.coef_, ord=1)

print(res_lasso_0.coef_)
print('The l2 norm of the lasso(a=0) coefficients is {0:5.1f}.'.format(coef_norm_l0))

In [ ]:
res_lasso_2 = linear_model.Lasso(alpha = 2.0).fit(X, base['Salary'])
coef_norm_2 = np.linalg.norm(res_lasso_2.coef_, ord=1)

lassos = pd.DataFrame({'var':var_list, 'lasso 2': res_lasso_2.coef_}) 
print(lassos)
print('\nThe l1 norm of the lasso(a=2) coefficients is {0:5.1f}.'.format(coef_norm_2))

## Practice
Take a few minutes and try the following. Feel free to chat with those around if you get stuck. The TA and I are here, too.

1. Try estimating a lasso regression when $\alpha = 5$. Compare the coefficient vector to the lasso with $\alpha=2$ from above.

2. Estimate lasso regressions with $\alpha$ equal to 10 and 200. What is happening to the coefficients? What is happening to the norm of the coefficients? Does this make sense? 

3. Use the `LassoCV()` function [(docs)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV) to estimate lasso regressions for the following grid of alphas:
```python
alpha_grid = [1, 1.5, 2, 3, 4, 5, 6, 8, 10]
```
Set the `cv` parameter to 10. This means the method will use k-fold cross validation with k=10.

4. Which $\alpha$ worked best? 
5. Use the `.mse_path_` attribute of the object returned by `LassoCV()` to return a (9,10) sized array. The 9 corresponds to the number of alphas and the 10 corresponds to the test mses from the 10-fold cross validation the function is using. 

6. Create a plot with $\alpha$ on the x-axis and the average test mse on the y-axis. To make sure the mse correspond to the correct alpha, use the `.alphas_` attribute of the results object for the x values. 

7. Now let the computer sort out the best tuning parameter. Call `LassoCV()` as before, but do not use the 'alphas' argument. The algorithm will search for the best $\alpha$.  

What is the optimal $\alpha$? 

What $\alpha$'s did the algorithm try? \[Use the `.alphas_` attribute.\]

8. Using the `mse_path_` and `alphas_` attributes, create a plot with $\alpha$ on the x-axis and the average test mse on the y-axis.

9. Display the optimal coefficient vector. Did the lasso eliminate any variables? 